# Bekerja dengan Komputasi

Saat Anda menjalankan skrip sebagai eksperimen Azure Machine Learning, Anda perlu menentukan konteks eksekusi untuk eksekusi eksperimen. Konteks eksekusi terdiri dari:

* Lingkungan Python untuk skrip, yang harus menyertakan semua paket Python yang digunakan dalam skrip.
* Target komputasi tempat skrip akan dijalankan. Ini bisa berupa stasiun kerja lokal tempat eksekusi eksperimen dijalankan, atau target komputasi jarak jauh seperti kluster pelatihan yang disediakan sesuai permintaan.

Dalam buku catatan ini, Anda akan menjelajahi *lingkungan* dan *target komputasi* untuk eksperimen.

## Menghubungkan ke ruang kerja Anda

Untuk memulai, hubungkan ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Mempersiapkan data untuk eksperimen

Dalam buku catatan ini, Anda akan menggunakan himpunan data yang berisi detail pasien diabetes. Jalankan sel di bawah ini untuk membuat himpunan data ini (jika sudah ada, kode akan menemukan versi yang ada)

In [ ]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'diabetes-data/')
                              )

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## Membuat skrip pelatihan

Jalankan dua sel berikut untuk membuat:

1. Folder untuk eksperimen baru
2. File skrip pelatihan yang menggunakan **scikit-learn** untuk melatih model dan **matplotlib** untuk membuat plot kurva ROC.

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Set regularization hyperparameter
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## Menentukan lingkungan

Saat Anda menjalankan skrip Python sebagai eksperimen di Azure Machine Learning, lingkungan Conda dibuat untuk menentukan konteks eksekusi skrip. Azure Machine Learning menyediakan lingkungan default yang mencakup banyak paket umum; termasuk paket **azureml-defaults** yang berisi pustaka yang diperlukan untuk bekerja dengan eksekusi eksperimen, serta paket populer seperti **pandas** dan **numpy**.

Anda juga dapat menentukan lingkungan Anda sendiri dalam file spesifikasi Conda, menambahkan paket menggunakan **conda** atau **pip** untuk memastikan eksperimen Anda memiliki akses ke semua pustaka yang diperlukan.

> **Catatan**: Dependensi conda diinstal terlebih dahulu, diikuti oleh dependensi pip. Karena paket **pip** diperlukan untuk menginstal dependensi pip, sebaiknya sertakan dalam dependensi conda.

Jalankan sel berikut untuk membuat file spesifikasi Conda bernama *experiment_env.yml* di folder yang sama dengan buku catatan ini.

In [ ]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Sekarang Anda dapat menggunakan file spesifikasi conda kustom untuk membuat lingkungan untuk eksperimen Anda

In [ ]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False 

# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

Sekarang Anda dapat menggunakan lingkungan untuk menjalankan skrip sebagai eksperimen.

Kode berikut menetapkan lingkungan yang Anda buat ke ScriptRunConfig, dan mengirimkan eksperimen. Saat eksperimen berjalan, amati detail eksekusi di widget dan di log output **azureml_logs/60_control_log.txt**, Anda akan melihat lingkungan conda sedang dibangun.

> **Catatan**: Kode di bawah membuat **DockerConfiguration** untuk eksekusi skrip, dan mengatur atribut **use_docker** ke **True** untuk menghosting lingkungan skrip dalam kontainer Docker. Ini adalah perilaku default, jadi Anda dapat menghilangkan perilaku ini; tetapi kami menyertakan perilaku default di sini agar lebih eksplisit.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                                environment=experiment_env,
                                docker_runtime_config=DockerConfiguration(use_docker=True)) # Use docker to host environment

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

Eksperimen berhasil menggunakan lingkungan, yang menyertakan semua paket yang diperlukan - Anda dapat melihat metrik dan output dari eksekusi eksperimen di Studio Azure Machine Learning, atau dengan menjalankan kode di bawah ini - termasuk model yang dilatih menggunakan **scikit-learn ** dan gambar bagan ROC yang dihasilkan menggunakan **matplotlib**.

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## Mendaftarkan lingkungan

Setelah mengalami kesulitan dalam menentukan lingkungan dengan paket yang Anda butuhkan, Anda dapat mendaftarkan lingkungan di ruang kerja.

In [ ]:
# Register the environment
experiment_env.register(workspace=ws)

Perhatikan bahwa lingkungan terdaftar dengan nama yang Anda tetapkan saat pertama kali membuat lingkungan (dalam kasus ini, *diabetes-experiment-env*).

Dengan lingkungan yang terdaftar, Anda dapat menggunakannya kembali untuk skrip apa pun yang memiliki persyaratan yang sama. Misalnya, mari kita buat folder dan skrip untuk melatih model diabetes menggunakan algoritma yang berbeda:

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Sekarang Anda dapat mengambil lingkungan terdaftar dan menggunakannya dalam eksperimen baru yang menjalankan skrip pelatihan alternatif (kali ini tidak ada parameter regularisasi karena pengklasifikasi Pohon Keputusan tidak memerlukannya).

In [ ]:
# get the registered environment
registered_env = Environment.get(ws, 'experiment_env')

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                              environment=registered_env,
                              docker_runtime_config=DockerConfiguration(use_docker=True)) # Use docker to host environment 

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

Kali ini eksperimen berjalan lebih cepat karena lingkungan yang cocok telah disimpan dari eksekusi sebelumnya, sehingga tidak perlu dibuat ulang di komputasi lokal. Tetapi, bahkan pada target komputasi yang berbeda, lingkungan yang sama akan dibuat dan digunakan - memastikan konsistensi untuk konteks eksekusi skrip eksperimen Anda.

Mari kita lihat metrik dan output dari eksperimen.

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## Melihat lingkungan terdaftar

Selain mendaftarkan lingkungan Anda sendiri, Anda dapat memanfaatkan lingkungan "yang dikumpulkan" yang dibuat sebelumnya untuk jenis eksperimen umum. Kode berikut mencantumkan semua lingkungan yang terdaftar:

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Name",env)

Semua lingkungan yang dikumpulkan memiliki nama yang dimulai ***AzureML-*** (Anda tidak dapat menggunakan prefiks ini untuk lingkungan Anda sendiri).

## Membuat kluster komputasi

Dalam banyak kasus, sumber daya komputasi lokal Anda mungkin tidak cukup untuk memproses eksperimen yang kompleks atau berjalan lama yang perlu memproses data dalam jumlah besar; dan Anda mungkin ingin memanfaatkan kemampuan untuk membuat dan menggunakan sumber daya komputasi secara dinamis di cloud. Azure Machine Learning mendukung berbagai target komputasi, yang dapat Anda tentukan di ruang kerja Anda dan gunakan untuk menjalankan eksperimen; membayar sumber daya hanya saat menggunakannya.

Anda dapat membuat kluster komputasi di [Studio Azure Machine Learning](https://ml.azure.com), atau dengan menggunakan SDK Azure Machine Learning. Sel kode berikut memeriksa ruang kerja Anda untuk keberadaan kluster komputasi dengan nama tertentu, dan jika tidak ada, Anda dapat membuat kluster komputasi.

> **Penting**: Ubah *your-compute-cluster* menjadi nama yang sesuai untuk kluster komputasi Anda dalam kode di bawah ini sebelum menjalankannya - Anda dapat menentukan nama kluster yang ada jika Anda memiliki kluster. Nama kluster harus berupa nama unik global dengan panjang antara 2 hingga 16 karakter. Karakter yang valid adalah huruf, angka, dan - karakter.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

> **Catatan**: Instans dan kluster komputasi didasarkan pada gambar mesin virtual Azure standar. Untuk latihan ini, gambar *Standard_DS11_v2* direkomendasikan untuk mencapai keseimbangan optimal antara biaya dan performa. Jika langganan Anda memiliki kuota yang tidak menyertakan gambar ini, pilih gambar alternatif; tetapi perlu diingat bahwa gambar yang lebih besar mungkin memerlukan biaya lebih tinggi dan gambar yang lebih kecil mungkin tidak cukup untuk menyelesaikan tugas. Atau, minta administrator Azure Anda untuk memperbesar kuota Anda.

## Menjalankan eksperimen pada komputasi jarak jauh

Sekarang Anda siap untuk menjalankan kembali eksperimen yang Anda jalankan sebelumnya, tetapi kali ini di kluster komputasi yang Anda buat. 

> **Catatan**: Eksperimen akan memakan waktu lebih lama karena gambar kontainer harus dibuat dengan lingkungan conda, lalu node kluster harus dimulai dan gambar disebarkan sebelum skrip dapat dijalankan. Untuk eksperimen sederhana seperti skrip pelatihan diabetes, skrip ini mungkin tampak tidak efisien; tetapi bayangkan Anda perlu menjalankan eksperimen yang lebih kompleks yang membutuhkan waktu beberapa jam - membuat komputasi yang lebih scalable secara dinamis dapat mengurangi keseluruhan waktu secara signifikan.

In [ ]:
# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=registered_env,
                                compute_target=cluster_name) 

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()

Sambil menunggu eksperimen berjalan, Anda dapat memeriksa status komputasi di widget di atas atau di [Studio Azure Machine Learning](https://ml.azure.com). Anda juga dapat memeriksa status komputasi menggunakan kode di bawah ini.

In [ ]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

Perhatikan bahwa perlu beberapa saat sebelum status berubah dari *menyeimbangkan* ke *mengubah ukuran* (sekarang mungkin saat yang tepat untuk istirahat sejenak!). Untuk memblokir kernel hingga eksekusi selesai, jalankan sel di bawah ini.

In [ ]:
run.wait_for_completion()

Perhatikan indikator kernel di kanan atas halaman, ketika indikator berubah dari **&#9899;** menjadi **&#9711;**, kode telah selesai dijalankan.

Setelah eksperimen selesai, Anda bisa mendapatkan metrik dan file yang dihasilkan oleh eksekusi eksperimen. Sekarang, file akan menyertakan log untuk membuat gambar dan mengelola komputasi.

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Sekarang Anda dapat mendaftarkan model yang dilatih oleh eksperimen.

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Compute cluster'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **Informasi Selengkapnya**:
>
> - Untuk informasi selengkapnya tentang lingkungan di Azure Machine Learning, lihat [Membuat & menggunakan lingkungan perangkat lunak di Azure Machine Learning](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments)
> - Untuk informasi selengkapnya tentang target komputasi di Azure Machine Learning, lihat [Apa yang dimaksud dengan target komputasi di Azure Machine Learning?](https://docs.microsoft.com/azure/machine-learning/concept-compute-target).